In [2]:
import glob
import librosa
import librosa.display
from matplotlib import pyplot as plt
import numpy as np
import os
import pystan
import scipy
import tqdm


subfolders = ["DCASE2013-pcen", "BirdVox-pcen", "SONYC-pcen"]
data_dir = '/beegfs/vl1019/spl2018_datasets'

%matplotlib inline

#sr=22050, hop_length=512, gain=0.98, bias=2, power=0.5,
#         time_constant=0.400, eps=1e-6, b=None, max_size=1, ref=None,
#axis=-1, max_axis=None

settings = {
    "fmin": 50,
    "fmax": 10000,
    "hop_length": 512,
    "n_fft": 2048,
    "n_mels": 128,
    "sr": 22050,
    "win_length": 1024,
    "window": "hann",
    "T": 0.400,
    "alpha": 0.98, #0.96,
    "delta": 2.0,
    "r": 0.5,
    "eps": 1e-6} #1e-10

In [3]:
is_fig_saved = False
subfolder_id = 1

logE_list = []
PCEN_list = []
EoverM_list = []
EoverMplusEps_list = []
G_list = []

subfolder_str = subfolders[subfolder_id]
subfolder_dir = os.path.join(data_dir, subfolder_str)
wav_names = list(glob.glob(os.path.join(subfolder_dir, "*.wav")))
wav_paths = sorted([os.path.join(subfolder_dir, x) for x in wav_names])
n_files = len(wav_paths)

for file_id in tqdm.tqdm(range(n_files)):
    wav_path = wav_paths[file_id]
    waveform, fs = librosa.load(wav_path, sr=22050)

    # Compute Short-Term Fourier Transform (STFT).
    stft = librosa.stft(
        waveform,
        n_fft=settings["n_fft"],
        win_length=settings["win_length"],
        hop_length=settings["hop_length"],
        window=settings["window"])


    # Compute squared magnitude coefficients.
    abs2_stft = (stft.real*stft.real) + (stft.imag*stft.imag)


    # Gather frequency bins according to the Mel scale.
    E = librosa.feature.melspectrogram(
        y=None,
        S=abs2_stft,
        sr=settings["sr"],
        n_fft=settings["n_fft"],
        n_mels=settings["n_mels"],
        htk=True,
        fmin=settings["fmin"],
        fmax=settings["fmax"])    

    EoverM = librosa.pcen(
        S=E,
        time_constant=settings["T"],
        sr=settings["sr"],
        hop_length=settings["hop_length"],
        gain=1.0,
        power=1.0,
        bias=settings["delta"],
        eps=1e-30) 

    EoverMplusEps = librosa.pcen(
        S=E,
        time_constant=settings["T"],
        sr=settings["sr"],
        hop_length=settings["hop_length"],
        gain=1.0,
        power=1.0,
        bias=settings["delta"],
        eps=settings["eps"]) 

    G = librosa.pcen(
        S=E,
        time_constant=settings["T"],
        sr=settings["sr"],
        hop_length=settings["hop_length"],
        gain=settings["alpha"],
        power=1.0,
        bias=settings["delta"],
        eps=settings["eps"]) 

    PCEN = librosa.pcen(
        S=E,
        time_constant=settings["T"],
        sr=settings["sr"],
        hop_length=settings["hop_length"],
        gain=settings["alpha"],
        power=settings["r"],
        bias=settings["delta"],
        eps=settings["eps"])

    logE_list.append(np.log(E))
    EoverM_list.append(EoverM)
    EoverMplusEps_list.append(EoverMplusEps)
    G_list.append(G)
    PCEN_list.append(PCEN)

logE = np.concatenate(logE_list, axis=1)
EoverM = np.concatenate(EoverM_list, axis=1)
EoverMplusEps = np.concatenate(EoverMplusEps_list, axis=1)
G = np.concatenate(G_list, axis=1)
PCEN = np.concatenate(PCEN_list, axis=1)

logE_bn = (logE - np.mean(logE)) / np.std(logE)
EoverM_bn = (EoverM - np.mean(EoverM)) / np.std(EoverM)
EoverMplusEps_bn = (EoverMplusEps - np.mean(EoverMplusEps)) / np.std(EoverMplusEps)
G_bn = (G - np.mean(G)) / np.std(G)
PCEN_bn = (PCEN - np.mean(PCEN)) / np.std(PCEN)

100%|██████████| 15/15 [00:34<00:00,  2.23s/it]


In [5]:
import pystan
import numpy as np

ocode = """
data {
    int<lower=1> N;
    real y[N];
}
parameters {
    real mu;
}
model {
    y ~ normal(mu, 1);
}
"""
sm = pystan.StanModel(model_code=ocode)
y2 = np.random.normal(size=20)
np.mean(y2)

op = sm.optimizing(data=dict(y=y2, N=len(y2)))

op

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_3aaa1aff3be33470f8a5bfa56085d51c NOW.


OrderedDict([('mu', array(-0.20243788863522433))])